In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
    apply_theme,
)
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
pre_df = pd.read_csv(join("..", "data", "Phase2.1", "temporal", "charlson", "charlson_countrylevel_preadmission.csv"))
post_df = pd.read_csv(join("..", "data", "Phase2.1", "temporal", "charlson", "charlson_countrylevel_sinceadmission.csv"))

pre_df["status"] = "Pre-admission"
post_df["status"] = "Post-admission"

country_sort = ["France", "Germany", "Italy", "Spain", "USA"]

df = pre_df.append(post_df)

df = df.loc[~df["country"].isin(["SINGAPORE"])]
df["country"] = df["country"].apply(lambda name: name if name in ["USA", "UK"] else name[0] + name.lower()[1:])
df["setting"] = df["setting"].apply(lambda s: s[0].upper() + s[1:])

orig_df = df

cs_df = pd.DataFrame()
for (country, setting, status), country_setting_df in df.groupby(by=["country", "setting", "status"]):    
    cs_df = cs_df.append({
        "country": country,
        "setting": setting,
        "status": status,
        "charlson_early": country_setting_df.loc[country_setting_df["wave"] == "early"].iloc[0]["charlson"],
        "charlson_late": country_setting_df.loc[country_setting_df["wave"] == "late"].iloc[0]["charlson"]
    }, ignore_index=True)
df = cs_df
df["difference"] = df["charlson_late"] - df["charlson_early"]
df["is_left_direction"] = df["difference"] < 0.0
df["is_right_direction"] = df["difference"] > 0.0
df["min"] = df.apply(lambda row: min(row["charlson_late"], row["charlson_early"]), axis='columns')
df.head()

In [ ]:
country_color_scale = alt.Scale(domain=["France", "Germany", "Italy", "Spain", "USA"], range=['#0072B2', '#E79F00', '#029F73', '#D45E00', '#CB7AA7'])

In [ ]:
severe_df = df.loc[df["setting"] == "All"]

index_domain = [severe_df[["charlson_early", "charlson_late"]].min().min() - 1, severe_df[["charlson_early", "charlson_late"]].max().max() + 1]

# Sort countries by their early wave index value, ascending

country_shift_sort = country_sort

triangle_opacity_scale = scale=alt.Scale(domain=[True, False], range=[1, 0])

status_sort = ["Pre-admission", "Post-admission"]


left_bars = alt.Chart().mark_bar(size=14, cornerRadiusTopLeft=9, cornerRadiusBottomLeft=9).encode(
    x=alt.X("charlson_early:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
    x2=alt.X2("charlson_late:Q"),
    y=alt.Y("status:N", axis=alt.Axis(title="Status"), scale=alt.Scale(domain=status_sort)),
    color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
    opacity=alt.Opacity("is_left_direction:Q", scale=triangle_opacity_scale, legend=None)
).properties(height=60)

right_bars = alt.Chart().mark_bar(size=14, cornerRadiusTopRight=9, cornerRadiusBottomRight=9).encode(
    x=alt.X("charlson_early:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
    x2=alt.X2("charlson_late:Q"),
    y=alt.Y("status:N", axis=alt.Axis(title="Status"), scale=alt.Scale(domain=status_sort)),
    color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
    opacity=alt.Opacity("is_right_direction:Q", scale=triangle_opacity_scale, legend=None)
)

chart = alt.layer(left_bars, right_bars, data=severe_df).facet(
    row=alt.Row("country:N", sort=country_shift_sort)
)

chart = chart.properties(
    title={
        "text": "Charlson index from early wave to late wave",
        "subtitle": "Severity: All | Status: Pre-admission",
        "subtitleColor": "gray",
        "dx": 91,
    }
)

chart = apply_theme(chart, title_anchor='start')

chart

In [ ]:
def make_charlson_arrow_by_country_chart(severity):
    severe_df = df.loc[df["setting"] == severity]

    index_domain = [severe_df[["charlson_early", "charlson_late"]].min().min() - 1, severe_df[["charlson_early", "charlson_late"]].max().max() + 1]

    # Sort countries by their early wave index value, ascending

    country_shift_sort = country_sort

    triangle_opacity_scale = scale=alt.Scale(domain=[True, False], range=[1, 0])

    status_sort = ["Pre-admission", "Post-admission"]

    bars = alt.Chart().mark_bar(size=6, opacity=1).encode(
        x=alt.X("charlson_early:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
        x2=alt.X2("charlson_late:Q"),
        y=alt.Y("status:N", axis=alt.Axis(title="Status"), scale=alt.Scale(domain=status_sort)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"))
    ).properties(height=50)

    left_points = alt.Chart().mark_point(shape="triangle-left", size=168, filled=True, opacity=1).encode(
        x=alt.X("charlson_late:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
        y=alt.Y("status:N", axis=alt.Axis(title="Status"), scale=alt.Scale(domain=status_sort)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
        opacity=alt.Opacity("is_left_direction:Q", scale=triangle_opacity_scale, legend=None)
    )

    right_points = alt.Chart().mark_point(shape="triangle-right", size=168, filled=True, opacity=1).encode(
        x=alt.X("charlson_late:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
        y=alt.Y("status:N", axis=alt.Axis(title="Status"), scale=alt.Scale(domain=status_sort)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
        opacity=alt.Opacity("is_right_direction:Q", scale=triangle_opacity_scale, legend=None)
    )

    chart = alt.layer(bars, left_points, right_points, data=severe_df).facet(
        row=alt.Row("country:N", header=alt.Header(title="Country"), sort=country_shift_sort)
    )

    chart = chart.properties(
        title={
            "text": "Charlson index from early wave to late wave",
            "subtitle": f"Severity: {severity}",
            "subtitleColor": "gray",
            "dx": 189,
        }
    )

    chart = apply_theme(chart, title_anchor='start')

    return chart

In [ ]:
make_charlson_arrow_by_country_chart("All")

In [ ]:
make_charlson_arrow_by_country_chart("Ever")

In [ ]:
def make_charlson_arrow_by_status_chart(severity):
    severe_df = df.loc[df["setting"] == severity]

    index_domain = [severe_df[["charlson_early", "charlson_late"]].min().min() - 1, severe_df[["charlson_early", "charlson_late"]].max().max() + 1]

    # Sort countries by their early wave index value, ascending

    country_shift_sort = country_sort

    triangle_opacity_scale = scale=alt.Scale(domain=[True, False], range=[1, 0])

    status_sort = ["Pre-admission", "Post-admission"]

    bars = alt.Chart().mark_bar(size=6, opacity=1).encode(
        x=alt.X("charlson_early:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
        x2=alt.X2("charlson_late:Q"),
        y=alt.Y("country:N", axis=alt.Axis(title="Country"), scale=alt.Scale(domain=country_shift_sort)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale)
    ).properties(height=120)

    left_points = alt.Chart().mark_point(shape="triangle-left", size=168, filled=True, opacity=1).encode(
        x=alt.X("charlson_late:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
        y=alt.Y("country:N", axis=alt.Axis(title="Country"), scale=alt.Scale(domain=country_shift_sort)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
        opacity=alt.Opacity("is_left_direction:Q", scale=triangle_opacity_scale, legend=None)
    )

    right_points = alt.Chart().mark_point(shape="triangle-right", size=168, filled=True, opacity=1).encode(
        x=alt.X("charlson_late:Q", scale=alt.Scale(domain=index_domain), axis=alt.Axis(title="Charlson index")),
        y=alt.Y("country:N", axis=alt.Axis(title="Country"), scale=alt.Scale(domain=country_shift_sort)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
        opacity=alt.Opacity("is_right_direction:Q", scale=triangle_opacity_scale, legend=None)
    )

    chart = alt.layer(bars, left_points, right_points, data=severe_df).facet(
        row=alt.Row("status:N", header=alt.Header(title="Status"), sort=status_sort)
    )

    chart = chart.properties(
        title={
            "text": "Charlson index from early wave to late wave",
            "subtitle": f"Severity: {severity}",
            "subtitleColor": "gray",
            "dx": 151,
        }
    )

    chart = apply_theme(chart, title_anchor='start')

    return chart

In [ ]:
make_charlson_arrow_by_status_chart("All")

In [ ]:
make_charlson_arrow_by_status_chart("Ever")

In [ ]:
def make_charlson_angle_chart(severity):
    all_df = df.loc[df["setting"] == severity].copy()
    all_orig_df = orig_df.loc[orig_df["setting"] == severity].copy()
    all_orig_df["wave"] = all_orig_df["wave"].apply(lambda w: w[0].upper() + w[1:])

    country_color_scale = alt.Scale(domain=["France", "Germany", "Italy", "Spain", "USA"], range=['#0072B2', '#E79F00', '#029F73', '#D45E00', '#CB7AA7'])

    # Sort countries by their early wave index value, ascending
    status_sort = ["Pre-admission", "Post-admission"]
    country_shift_sort = country_sort

    lines = alt.Chart().mark_line(size=4).encode(
        y=alt.Y("charlson:Q", axis=alt.Axis(title="Charlson index")),
        x=alt.X("wave:N", axis=alt.Axis(title=None)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
        strokeDash=alt.StrokeDash("status:N", legend=alt.Legend(title="Admission Status", symbolType="stroke"), scale=alt.Scale(domain=status_sort, range=[[4, 4], [1,0]]))
    ).properties(width=70)
    
    points = alt.Chart().mark_point(filled=True, size=40, opacity=1).encode(
        y=alt.Y("charlson:Q", axis=alt.Axis(title="Charlson index")),
        x=alt.X("wave:N", axis=alt.Axis(title=None)),
        color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
    )

    chart = alt.layer(lines, points, data=all_orig_df).facet(
        column=alt.Column("country:N", header=alt.Header(title="Country", orient='bottom')),
    )

    chart = chart.properties(
        title={
            "text": "Charlson index from early wave to late wave",
            "subtitle": f"Severity: {severity}",
            "subtitleColor": "gray",
            "dx": 53,
        }
    )

    chart = apply_theme(chart, title_anchor='start')

    return chart

In [ ]:
make_charlson_angle_chart("All")

In [ ]:
make_charlson_angle_chart("Ever")